In [0]:
!nvidia-smi

Sun Apr 26 07:59:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# @title Import Packages and other Config

!pip install -qq transformers

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import os
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%pip install wandb -q
import wandb
wandb.login()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

     |████████████████████████████████| 573kB 2.7MB/s 
     |████████████████████████████████| 890kB 12.6MB/s 
     |████████████████████████████████| 1.0MB 15.9MB/s 
     |████████████████████████████████| 3.7MB 25.7MB/s 
     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 460kB 14.5MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 112kB 19.0MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


###Download the IMDB Reviews dataset

In [0]:
%%capture
if not os.path.isdir('aclImdb'):
    !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar -xvf 'aclImdb_v1.tar.gz'
    !sudo rm -r 'aclImdb_v1.tar.gz'

## Generate Dataset

In [0]:
class IMDB_dataset(Dataset):
    
    def __init__(self,root_dir, seq_length = 250):
        
        self.reviews = []
        self.labels = []
        self.seq_length = seq_length
        self.reviews_len = []

        for filename in os.listdir(os.path.join(root_dir,'pos')):
            review = open(os.path.join(os.path.join(root_dir,'pos'),filename),'r').read().replace('<br />',' ')
            self.reviews.append(review)
            self.labels.append(1)

        for filename in os.listdir(os.path.join(root_dir,'neg')):
            review = open(os.path.join(os.path.join(root_dir,'neg'),filename),'r').read().replace('<br />',' ')
            self.reviews.append(review)
            self.labels.append(0)

        self.reviews_len = [len(review.split()) for review in self.reviews]

    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self,index):  
        # encode the reviews
        encoding = tokenizer.encode_plus(
            self.reviews[index],
            add_special_tokens=True,
            max_length=self.seq_length,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        data_dict = {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'targets': torch.tensor(self.labels[index], dtype=torch.float).unsqueeze(-1)}
        return data_dict

    def get_reviews(self,index):
        return self.reviews[index] 

    def analyze_reviews(self):
        print('Max: {}'.format(max(self.reviews_len)))
        print('Min: {}'.format(min(self.reviews_len)))
        print('Mean: {}'.format(np.mean(self.reviews_len)))
        
dataset = IMDB_dataset('aclImdb/train', seq_length = 250)

print('Total number of Reviews: {}'.format(len(dataset)))
print('Number of Positives: {}'.format(dataset.labels.count(0)))
print('Number of Negatives: {}'.format(dataset.labels.count(1)))

dataset.analyze_reviews()

Total number of Reviews: 25000
Number of Positives: 12500
Number of Negatives: 12500
Max: 2459
Min: 10
Mean: 231.48692


In [0]:
data = dataset[100]
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'])
print(dataset.get_reviews(100))
print(tokenizer.convert_ids_to_tokens(data['input_ids']))

torch.Size([250])
torch.Size([250])
tensor([1.])
Didn't Mystic Pizza win the Oscar for that year? This movie never had a chance, due to the casting, but perhaps by now people can see why I felt that way upon leaving the theater. Only "Wargames" left me feeling similarly disturbed after leaving the theater, with the feeling I was getting a glimpse of our future. History has shown that this is exactly what happened.  The casting is a pop-culture Cuisinart of the 1980s: you get Arnold as Ben Richards, the framed fugitive offered a chance to "run" for his freedom on the game show with the same title as the movie; Richard Dawson as Damon Killian, treating his role as if he were the host of Family Feud with the contestants using real guns; Jesse Ventura as "Captain Freedom" and co-wrestler Professor Tanaka as "Subzero," both "stalkers" who kill the "running men." Even Mick Fleetwood (Mic) and Dweezil Zappa (Stevie) show up, while the "dancers of the future" are none other than the Laker Girl

## Generate Dataloaders

In [0]:
# split the dataset into validation and test sets
len_valid_set = int(0.1*len(dataset))
len_train_set = len(dataset) - len_valid_set

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_set))

train_dataset , valid_dataset = torch.utils.data.random_split(dataset , [len_train_set, len_valid_set])

batch_size = 32

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

batch = next(iter(train_loader))
print(batch['input_ids'].shape)
print(batch['attention_mask'].shape)
print(batch['targets'].shape)

The length of Train set is 22500
The length of Valid set is 2500
torch.Size([32, 250])
torch.Size([32, 250])
torch.Size([32, 1])


## Network (BERT)

In [0]:
class SentimentClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        hidden_state, output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.dropout(output)
        output = self.classifier(output)
        output = self.sigmoid(output)
        return output

network = SentimentClassifier().to(device)

## Helper functions for training 

In [0]:
def print_overwrite(step, total_step, loss, acc, operation):
    sys.stdout.write('\r')
    if operation == 'train':
        sys.stdout.write("Train Steps: %d/%d  Loss: %.4f  Accuracy: %.4f" % (step, total_step, loss, acc))
    else:
        sys.stdout.write("Valid Steps: %d/%d  Loss: %.4f  Accuracy: %.4f" % (step, total_step, loss, acc))
    sys.stdout.flush()

# function to find the accuracy
def find_acc(pred, label):
    correct = torch.round(pred).eq(label)
    accuracy = correct.to(torch.float32).mean().item() * 100
    return accuracy

def train(network, criterion, optimizer, scheduler):
    network = network.train()
    loss = []
    accuracy = []
    
    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        targets = batch["targets"].to(device)

        logits = network(input_ids=input_ids, attention_mask=attention_mask)

        optimizer.zero_grad()

        loss_step = criterion(logits, targets)
        acc_step = find_acc(logits, targets)

        loss_step.backward()
        nn.utils.clip_grad_norm_(network.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        print_overwrite(step, len(train_loader), loss_step, acc_step, 'train')

        loss.append(loss_step.item())
        accuracy.append(acc_step)

    return np.mean(loss), np.mean(accuracy)

def validate(network, criterion):
    network = network.eval()
    loss = []
    accuracy = []
    
    with torch.no_grad():
        for step, batch in enumerate(valid_loader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            targets = batch["targets"].to(device)

            logits = network(input_ids=input_ids, attention_mask=attention_mask)

            loss_step = criterion(logits, targets)
            acc_step = find_acc(logits, targets)

            print_overwrite(step, len(valid_loader), loss_step, acc_step, 'valid')

            loss.append(loss_step.item())
            accuracy.append(acc_step)

    return np.mean(loss), np.mean(accuracy)

## Training Loop

To reproduce the training procedure from the BERT paper, we'll use the [AdamW](https://huggingface.co/transformers/main_classes/optimizer_schedules.html#adamw) optimizer provided by Hugging Face. It corrects weight decay, so it's similar to the original paper. We'll also use a linear scheduler with no warmup steps.

The BERT authors have some recommendations for fine-tuning:

- Batch size: 16, 32
- Learning rate (Adam): 5e-5, 3e-5, 2e-5
- Number of epochs: 2, 3, 4

In [0]:
num_epochs = 10
acc_max = 0

# WandB – Initialize a new run
wandb.init(name='BERT', entity="arkalim", project="personal")

optimizer = AdamW(network.parameters(), lr=2e-5, correct_bias=False)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps= len(train_loader) * num_epochs)

criterion = nn.BCELoss().to(device)

for epoch in range(1,num_epochs+1):

    loss_train, acc_train = train(network, criterion, optimizer, scheduler)
    loss_valid, acc_valid = validate(network, criterion)
        
    sys.stdout.write('\r')
    sys.stdout.flush()
    print('\n----------------------------------------------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Train Acc: {:.4f}  Valid Loss: {:.4f}  Valid Acc: {:.4f}'.format(epoch, loss_train, acc_train, loss_valid, acc_valid))
    print('----------------------------------------------------------------------------------------')

    if acc_valid > acc_max:
        acc_max = acc_valid
        torch.save(network.state_dict(), 'sentiment_analysis_bert.pth') 
        print("\nMaximum validation Acc of {:.4f} at epoch {}/{}".format(acc_max, epoch, num_epochs))
        print('Model Saved\n')
    
    wandb.log({
        "Epoch": epoch,
        "Train Loss": loss_train,
        "Train Acc": acc_train,
        "Valid Loss": loss_valid,
        "Valid Acc": acc_valid,
        "Max Valid Acc": acc_max})


----------------------------------------------------------------------------------------
Epoch: 1  Train Loss: 0.2979  Train Acc: 87.5932  Valid Loss: 0.2598  Valid Acc: 89.9525
----------------------------------------------------------------------------------------

Maximum validation Acc of 89.95253164556962 at epoch 1/10
Model Saved

----------------------------------------------------------------------------------------
Epoch: 2  Train Loss: 0.1378  Train Acc: 95.1660  Valid Loss: 0.2689  Valid Acc: 91.0997
----------------------------------------------------------------------------------------

Maximum validation Acc of 91.0996835443038 at epoch 2/10
Model Saved

----------------------------------------------------------------------------------------
Epoch: 3  Train Loss: 0.0687  Train Acc: 98.1357  Valid Loss: 0.3502  Valid Acc: 92.2468
----------------------------------------------------------------------------------------

Maximum validation Acc of 92.24683544303798 at epoch 3

KeyboardInterrupt: ignored

### Predicting on Raw Text

Let's use our model to predict the sentiment of some raw text:

In [0]:
best_network = SentimentClassifier().to(device)
best_network.load_state_dict(torch.load('sentiment_analysis_bert.pth'));

In [0]:
review_text = "The ending was amazing but the overall movie was very slow."

encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=160,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = best_network(input_ids, attention_mask)

# convert output probabilities to predicted class (0 or 1)
pred = torch.round(output.squeeze()).item()

# print custom response based on whether test_review is pos/neg
if(pred==1):
    print('Positive Review:  Confidence: {:.3f} %'.format((output.item())*100))
else:
    print('Negative Review:  Confidence: {:.3f} %'.format((1-output.item())*100))

Negative Review:  Confidence: 99.709 %
